# WEEK 2: Data Scraping Automation

## What is Data Scraping Automation
Data scraping automation is the process of using software to extract information from websites by replicating human-like actions such as clicking, scrolling, typing, and navigating pages.

## Core Behaviour
- Click
- Scroll
- Input
- Hover
- Delay
- Other

## Google Maps Data Scraping A
https://www.google.com/maps

Task: Retrieve data for places that serve ramen in Malang City.

Output: Title, address, and URLS

## Google Maps Data Scraping B
https://www.google.com/maps

Task: Retrieve data for places that serve nasi ayam within 1 km of Alun-Alun Kota Malang.

Output: Title, address, coordinate, and URLS

## Google Maps Data Scraping C (Challenge)
https://www.google.com/maps

Task: Retrieve all reviews from the Malang Strudel branch that is closest to Malang Town Square.

Output: Username, photos included in the review, user’s, posting time, star rating, and review text.

## 